In [34]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [43]:
from helpers import *
from implementations import *
from cross_validation import *
from helper_implementations import *
from standardization import *
from cleaning_data import *

In [49]:
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [50]:
print(y.shape)

(250000,)


## Calculate the weights with different methods

In [86]:
# Initialization
max_iters = 50
gamma = 0.3
lambda_=0.3
initial_w=np.zeros(30)
k_fold=4
#x=build_poly(tX,8)
max_degree=2
#print(x.shape)
#Pour output un dossier .csv, assurez vous d'avoir train.csv et test.csv dans le même dossier que ce fichier
#Remplacer weightsi par weights si vous voulez utiliser la méthode i (i \in {1,2,3,4})
#Lancer les deux dernières cellues, le dossier en output aura le titre submission.csv
#weights,_=least_squares_GD(y,x,initial_w,max_iters,gamma)
#weights,_=least_squares_SGD(y,tX,initial_w,max_iters,gamma)
#weights,_=least_squares(y,x)
#weights,_=ridge_regression(y,x,lambda_)
#degree,gamma=cross_validation_GD_SGD_LR(y, tX, k_fold,5,np.logspace(-3, -1, 20), max_iters,'SGD')
#degree,gamma=cross_validation_GD_SGD_LR(y, x, k_fold, max_degree,np.logspace(-8, -5, 20), max_iters,'LR')
#a=cross_validation_LS(y,tX,k_fold,8)
#d,l=cross_validation_RR(y, tX, k_fold, 10, np.logspace(-3, 0, 30))


In [56]:
def transform_x_least_squares(x, deg):
    #Replace the -999 in the first column by the mean (without the -999)
    x[:, 0][x[:, 0] == -999.0] = np.mean(x[:, 0][x[:, 0] != -999.0])
    
    #Split the data w.r.t. the number of jets
    x_0, x_1, x_23 = split_x(x)
    
    #Standardization
    x_0, x_0_mean, x_0_std = standardize(x_0)
    x_1, x_1_mean, x_1_std = standardize(x_1)
    x_23, x_23_mean, x_23_std = standardize(x_23)
    
    #Polynomial expansion
    x_0=build_poly(x_0,deg)
    x_1=build_poly(x_1,deg)
    x_23=build_poly(x_23,deg)
    
    #Adding ones to the first column
    tx_0 = np.c_[np.ones((x_0.shape[0], 1)), x_0]
    tx_1 = np.c_[np.ones((x_1.shape[0], 1)), x_1]
    tx_23 = np.c_[np.ones((x_23.shape[0], 1)), x_23]
    
    return tx_0, tx_1, tx_23

def transform_y_least_squares(y, x):
    
    #Split the data w.r.t. the number of jets 
    y = np.expand_dims(y, axis=1)
    y_0, y_1, y_23 = split_y(y, x)
    
    return y_0, y_1, y_23

In [87]:
tx_0,tx_1,tx_23=transform_x_least_squares(tX,1)
y_0,y_1,y_23=transform_y_least_squares(y,tX)


#Computing the weights and loss on the training set
w_0, loss_w0 = least_squares(y_0, tx_0)
w_1, loss_w1 = least_squares(y_1, tx_1)
w_23, loss_w2 = least_squares(y_23, tx_23)

x_t_0,x_t_1,x_t_23=transform_x_least_squares(tX,1)
print(x_t_0.shape)
print(x_t_1.shape)
print(x_t_23.shape)

#Predict weights
y_prediction_train = label_regression(w_0,w_1,w_23,x_t_0,x_t_1,x_t_23,tX)
print(y_prediction_train.shape)
print(ids_test.shape)

(99913, 19)
(77544, 23)
(72543, 31)
id_ (250000,)
y_0 (99913, 1)
ypred (250000,)
y (250000, 2)
(250000,)
(568238,)


In [82]:
def compute_accuracy(y, y_prediction):
    ''' computes accuracy of a prediction '''
    correct = 0.0
    for i in range (len(y)):
        if(y[i] == y_prediction[i]):
            correct += 1
    accuracy = correct/len(y)
    return accuracy

In [84]:
compute_accuracy(y, y_prediction_train)

0.75976

In [85]:
w,_=least_squares(y,tX)
y_pred = predict_labels(w,tX)
compute_accuracy(y, y_pred)

0.743976

## Generate predictions and save ouput in csv format for submission:

In [78]:
DATA_TEST_PATH = 'test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [20]:
x_test = standardize_x_te(tX_test,mean_x,std_x)
x_test=build_poly(x_test,8)

In [21]:
OUTPUT_PATH = 'submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, x_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [41]:
from numpy import asarray
from numpy import savetxt
savetxt('dtessst.csv',x, delimiter=',')